# pyspark (standalone cluster mode) is not running on jupyter!?

I have seen many people have issue with pyspark while running a cluster of spark in standalone mode, but it does not work when run your application in python script or notebook.

## why?
It seems when you have a cluster up and running and try to connect your spark application written in notebook to cluster, all the resources (workers) are not available and you will get the following error:

```Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources.```

But if you try the same code and run line-by-line sequentionally in pyspark-shell, it works.

## Solution

Why not force jupyter be like pyspark-shell.
I went through the source code of Spark till I get how ot settle the issue. You will see I tried least modification to let you know where the following originated.
I modified ```$SPARK_HOME/python/pyspark/shell.py```

the following line is what I added which makes Spark works on standalone cluster in jupyter notebook.

```os.environ['PYSPARK_SUBMIT_ARGS'] = '"--master" "spark://192.168.1.12:7077" "--name" "PySparkShell" "pyspark-shell"'```

replace ```"--master" "spark://192.168.1.12:7077"``` with your spark master address.

In [1]:
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

"""
An interactive shell.

This file is designed to be launched as a PYTHONSTARTUP script.
"""

import atexit
import os
import platform
import warnings
os.environ['PYSPARK_SUBMIT_ARGS'] = '"--master" "spark://192.168.1.12:7077" "--name" "PySparkShell" "pyspark-shell"'
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

if os.environ.get("SPARK_EXECUTOR_URI"):
    SparkContext.setSystemProperty("spark.executor.uri", os.environ["SPARK_EXECUTOR_URI"])
    print(os.environ["SPARK_EXECUTOR_URI"])

# print(os.environ)
SparkContext._ensure_initialized()  # type: ignore
try:
    spark = SparkSession._create_shell_session()  # type: ignore
except Exception:
    import sys
    import traceback
    warnings.warn("Failed to initialize Spark session.")
    traceback.print_exc(file=sys.stderr)
    sys.exit(1)

sc = spark.sparkContext
sql = spark.sql
atexit.register(lambda: sc.stop())

# for compatibility
sqlContext = spark._wrapped
sqlCtx = sqlContext

print(r"""Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version %s
      /_/
""" % sc.version)
print("Using Python version %s (%s, %s)" % (
    platform.python_version(),
    platform.python_build()[0],
    platform.python_build()[1]))
print("Spark context Web UI available at %s" % (sc.uiWebUrl))
print("Spark context available as 'sc' (master = %s, app id = %s)." % (sc.master, sc.applicationId))
print("SparkSession available as 'spark'.")

# The ./bin/pyspark script stores the old PYTHONSTARTUP value in OLD_PYTHONSTARTUP,
# which allows us to execute the user's PYTHONSTARTUP file:
_pythonstartup = os.environ.get('OLD_PYTHONSTARTUP')
if _pythonstartup and os.path.isfile(_pythonstartup):
    with open(_pythonstartup) as f:
        code = compile(f.read(), _pythonstartup, 'exec')
        exec(code)

#######################################%%%%#########################
/usr/lib/jvm/java-1.8.0-openjdk-amd64/bin/java -cp /home/moses/spark/conf/:/home/moses/spark/jars/* -Xmx1g org.apache.spark.deploy.SparkSubmit --master spark://192.168.1.12:7077 --name PySparkShell pyspark-shell


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/22 12:59:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.2.0
      /_/

Using Python version 3.9.5 (default, Jun  4 2021 12:28:51)
Spark context Web UI available at http://192.168.1.12:4040
Spark context available as 'sc' (master = spark://192.168.1.12:7077, app id = app-20220222125957-0004).
SparkSession available as 'spark'.


In [3]:
sc.parallelize(range(10)).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]